In [ ]:
!pip install --upgrade gensim

In [ ]:
import gensim.downloader as api
wv=api.load('word2vec-google-news-300')

In [ ]:
import pandas as pd
df=pd.read_csv("fake_and_real_news.csv")
df.shape

(9900, 2)

In [ ]:
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [ ]:
df.label.value_counts()

Fake    5000
Real    4900
Name: label, dtype: int64

In [ ]:
df["label_num"]=df.label.map({"Fake":0,"Real":1})

In [ ]:
import spacy
!python -m spacy download en_core_web_lg
nlp=spacy.load("en_core_web_lg")



2023-10-06 10:44:25.975576: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
def preprocess_and_vectorize(text):
  filtered_token=[]
  doc=nlp(text)
  for token in doc:
    if token.is_stop and token.is_punct:
      continue
    filtered_token.append(token.lemma_)
  return wv.get_mean_vector(filtered_token)

In [ ]:
df["pv"]=df.Text.apply(lambda text: preprocess_and_vectorize(text))

In [ ]:
from sklearn.model_selection import train_test_split
xtr,xte,ytr,yte=train_test_split(df.pv,df.label_num,test_size=0.2,stratify=df.label_num)

In [ ]:
import numpy as np
xtr_2d=np.stack(xtr)
xte_2d=np.stack(xte)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

clf=GradientBoostingClassifier()
clf.fit(xtr_2d,ytr)
y_preds=clf.predict(xte_2d)
print(classification_report(y_preds,yte))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1000
           1       0.98      0.98      0.98       980

    accuracy                           0.98      1980
   macro avg       0.98      0.98      0.98      1980
weighted avg       0.98      0.98      0.98      1980



In [ ]:
test_news=["A team of scientists has developed a new vaccine that prevents all types of cancer, according to a study published in the journal Science. The vaccine works by targeting a protein that is found on the surface of all cancer cells. The study found that the vaccine was effective in preventing cancer in mice and rats."]
test_news_vectors=[preprocess_and_vectorize(n) for n in test_news]
clf.predict(test_news_vectors)

array([1])